# Project Context and Problem Definition
## Non-Technical Loss Detection in Electricity Distribution Networks

**Document Type:** Executive Overview and System Design Rationale  
**Audience:** Utility Executives, Regulatory Auditors, Data Science Leadership  
**Version:** 1.0  

---

## 1. Executive Summary

### The Problem

Non-Technical Losses (NTL) represent electricity consumed but not billed, primarily due to:
- **Meter tampering and bypass** (physical intervention)
- **Billing manipulation** (administrative fraud)
- **Illegal connections** (unauthorized grid access)
- **Data corruption** (transmission errors, meter malfunction)

Global utilities lose an estimated **USD 96 billion annually** to NTL (World Bank, 2019). In developing markets, NTL can exceed 20% of total energy distributed, directly impacting:
- Revenue assurance and financial stability
- Cross-subsidy distortions (honest customers pay more)
- Grid planning accuracy (demand forecasting errors)
- Regulatory compliance and public trust

### Why Traditional Methods Fail

| Approach | Limitation |
|----------|------------|
| Energy balance audits | Only detect aggregate loss, not individual actors |
| Threshold-based flags | High false positive rates; easily circumvented |
| Random inspections | Low hit rate (~5-8%); expensive per detection |
| Manual analysis | Does not scale; inconsistent enforcement |

### This System

This project implements a **hybrid detection pipeline** combining:
1. **Customer segmentation** to establish behavioral baselines
2. **Unsupervised anomaly detection** for pattern-based screening
3. **Supervised classification** where labeled data exists
4. **Explainable scoring** for inspector prioritization

The system is designed for **human-in-the-loop operation**: it generates prioritized investigation lists, not automatic disconnections.

### System Architecture Overview

```
                                EINSTEIN NTL Detection System
    ============================================================================
    
    [Smart Meter Data]      [Historical Labels]       [External Data]
           |                       |                        |
           v                       v                        v
    +---------------------------------------------------------------------------+
    |                        DATA INGESTION LAYER                               |
    |   - Schema validation       - Temporal alignment      - Quality scoring   |
    +---------------------------------------------------------------------------+
                                       |
                                       v
    +---------------------------------------------------------------------------+
    |                     FEATURE ENGINEERING PIPELINE                          |
    |   - Rolling statistics      - Anomaly indicators      - Calendar features |
    |   - Consumption ratios      - Behavioral patterns     - Quality flags     |
    +---------------------------------------------------------------------------+
                                       |
                                       v
    +---------------------------------------------------------------------------+
    |                      CUSTOMER SEGMENTATION                                |
    |   - Load profile clustering    - Behavioral grouping    - Risk stratification |
    +---------------------------------------------------------------------------+
                                       |
                                       v
    +---------------------------------------------------------------------------+
    |                       DETECTION MODELS                                    |
    |   Isolation Forest    |    XGBoost    |    Autoencoder                    |
    |   (per cluster)       |   (supervised)|    (sequence)                     |
    +---------------------------------------------------------------------------+
                                       |
                                       v
    +---------------------------------------------------------------------------+
    |                      ENSEMBLE & CALIBRATION                               |
    |   - Score fusion           - Confidence estimation      - Threshold tuning |
    +---------------------------------------------------------------------------+
                                       |
                                       v
    +---------------------------------------------------------------------------+
    |                      OPERATIONAL INTERFACE                                |
    |   REST API    |    Operations Dashboard    |    Audit Reports             |
    +---------------------------------------------------------------------------+
                                       |
                                       v
    +---------------------------------------------------------------------------+
    |                    HUMAN-IN-THE-LOOP WORKFLOW                             |
    |   Flag --> Review --> Investigate --> Confirm/Dismiss --> Feedback        |
    +---------------------------------------------------------------------------+
```

## 2. Problem Definition

### 2.1 What Constitutes "Theft"

For modeling purposes, we define electricity theft as **consumption patterns inconsistent with meter-registered usage that cannot be explained by legitimate behavioral changes**.

Key distinction: **Abnormal is not equivalent to fraudulent.**

| Scenario | Classification | Model Implication |
|----------|----------------|-------------------|
| Sudden 80% consumption drop, no meter issues | Suspicious | Flag for investigation |
| Seasonal vacation pattern (regular annual) | Normal | Learn seasonal baselines |
| New industrial equipment installation | Normal | Detect legitimate step change |
| Meter replaced, data gap followed by change | Ambiguous | Exclude from training |
| Consistent zero readings, building occupied | Suspicious | High-priority flag |

### 2.2 Why Naive Thresholding Fails

A common approach is: "Flag customers whose consumption dropped >50% month-over-month."

**Problems:**
1. **Seasonal variation:** Air conditioning load in summer vs. winter creates legitimate 50%+ swings
2. **Customer heterogeneity:** A 50% drop for a commercial flat-load customer is meaningful; for a vacation home, it is expected
3. **Adversarial adaptation:** Sophisticated actors reduce consumption gradually to avoid threshold triggers
4. **Data quality:** Meter resets, transmission gaps, and billing cycle shifts create false signals

### 2.3 Operational Cost of Errors

| Error Type | Operational Impact | Estimated Cost |
|------------|-------------------|----------------|
| **False Positive** | Wasted inspection, customer dispute, reputational damage | USD 150-500 per case |
| **False Negative** | Continued revenue loss, unfair burden on honest customers | USD 200-2000/month/customer |

The asymmetry varies by jurisdiction. In markets with strong consumer protection, false positives are particularly costly. This system emphasizes **precision at the top of the ranked list** (precision@k) over raw accuracy.

## 3. Data Landscape

### 3.1 Smart Meter Data Characteristics

Modern Advanced Metering Infrastructure (AMI) provides:
- **Temporal resolution:** 15-minute to hourly intervals
- **Coverage:** 90-98% of residential and commercial customers in deployed regions
- **Data dimensions:** Active/reactive power, voltage, power factor, tamper events

### 3.2 Data Quality Challenges

| Issue | Prevalence | Mitigation |
|-------|------------|------------|
| Missing readings | 5-15% of records | Seasonal median imputation with quality flags |
| Communication gaps | 2-8% of meters at any time | Gap duration tracking; exclude long gaps from training |
| Meter replacements | 3-5% annually | Detect discontinuities; reset baseline |
| Time zone / DST errors | Occasional | Validate temporal alignment |
| Negative readings | Rare (<0.1%) | Physical validation; clamp or exclude |

### 3.3 Label Quality

If historical labels exist (confirmed theft cases), they typically suffer from:
- **Selection bias:** Only inspected customers are labeled
- **Confirmation bias:** Inspectors find what they expect to find
- **Temporal mismatch:** Theft may have occurred months before detection

This motivates our hybrid approach: use supervised learning where labels are reliable, but maintain unsupervised detection capability for discovery.

## 4. Modeling Philosophy

### 4.1 Why Hybrid Detection

No single approach addresses all detection scenarios:

| Method | Strength | Weakness |
|--------|----------|----------|
| **Supervised (XGBoost)** | Learns from confirmed cases | Requires labels; misses novel patterns |
| **Unsupervised (Isolation Forest)** | Detects anomalies without labels | High false positive rate on heterogeneous customers |
| **Sequence (Autoencoder)** | Captures temporal patterns | Computationally expensive; requires tuning |
| **Rule-based** | Transparent, auditable | Rigid; easily circumvented |

Our ensemble combines these methods, weighted by confidence and validated against historical data.

### 4.2 Why Customer Clustering First

Customers exhibit fundamentally different consumption patterns:
- **Residential base-load:** Consistent 10-20 kWh/day with morning/evening peaks
- **Residential high-variance:** 5-50 kWh/day with seasonal HVAC swings
- **Commercial flat-load:** 100-500 kWh/day, 6-day pattern
- **Industrial continuous:** 500+ kWh/day, minimal variance

A single anomaly threshold applied globally would:
- Under-flag low-variance commercial customers (subtle theft masked by scale)
- Over-flag high-variance residential customers (legitimate volatility misclassified)

We first segment customers into behavioral clusters, then train cluster-specific models.

### 4.3 Why Explainability is Non-Negotiable

Regulatory and operational requirements demand:
1. **Audit trail:** Every flag must be traceable to specific data and model version
2. **Inspector guidance:** Field teams need actionable reasons, not just scores
3. **Dispute defense:** Customer challenges require evidence beyond "the model said so"
4. **Bias detection:** Unexplainable patterns may indicate protected-class discrimination

We implement SHAP-based explanations with fallback to permutation importance.

## 5. Ethical and Regulatory Considerations

### 5.1 Customer Privacy

Smart meter data reveals sensitive information:
- Occupancy patterns (when residents are home/away)
- Appliance usage (health equipment, entertainment habits)
- Economic status (consumption levels correlate with income)

**Mitigations implemented:**
- Customer IDs hashed before model training
- No raw consumption data in logs or reports
- Aggregated metrics only in operational dashboards
- Data retention policies aligned with regulatory requirements

### 5.2 Bias Risk

Theft detection models may exhibit bias:
- **Geographic:** Higher false positives in low-income areas with older infrastructure
- **Demographic:** Consumption patterns correlated with household composition
- **Seasonal:** Customers with irregular schedules flagged disproportionately

**Mitigations implemented:**
- Cluster-specific thresholds to normalize across customer types
- Regular audit of flag distribution by geographic and demographic segments
- Human review required before any enforcement action

### 5.3 Human-in-the-Loop Requirement

This system explicitly **does not:**
- Automatically disconnect customers
- Generate penalty bills without investigation
- Share individual predictions with third parties

All model outputs are **decision support** for trained investigators. The workflow enforces:
```
Model Flag --> Supervisor Review --> Field Investigation --> Evidence Collection --> Action
```

## 6. Success Metrics (Business-Aligned)

### 6.1 Primary Metrics

| Metric | Definition | Target |
|--------|------------|--------|
| **Precision@1%** | Proportion of actual theft in top 1% of scored customers | > 70% |
| **Precision@5%** | Proportion of actual theft in top 5% of scored customers | > 50% |
| **Time-to-Detection** | Days between theft onset and model flag | < 30 days (median) |
| **Inspector Efficiency** | Confirmed cases per 100 inspections | > 25 (baseline: 5-8) |

### 6.2 Operational Metrics

| Metric | Definition | Target |
|--------|------------|--------|
| **Monthly Flag Count** | Number of customers flagged for investigation | < 2% of customer base |
| **False Positive Rate (seasonal)** | FP rate should not spike in vacation/holiday periods | Variance < 20% |
| **Model Drift Alert** | PSI score indicating feature distribution shift | < 0.25 |
| **Score Latency** | Time to score single customer via API | < 200ms (p95) |

### 6.3 Revenue Impact (Estimated)

Assuming:
- 100,000 metered customers
- 3% actual theft rate
- Average theft value: USD 100/month/customer
- Current random inspection hit rate: 7%
- Model-guided inspection hit rate: 30%

**Estimated annual recovery improvement:** USD 1.2M - 2.5M  
**Estimated inspection cost savings:** USD 200K - 400K

## 7. Scope and Limitations

### 7.1 In Scope

- Detection of consumption-pattern anomalies indicative of NTL
- Customer segmentation for baseline establishment
- Explainable scoring and prioritization
- API-based integration with utility operations systems
- Audit logging and model governance artifacts

### 7.2 Out of Scope

- **Physical meter diagnostics:** Tamper detection at device level requires hardware integration
- **Billing system fraud:** Administrative fraud in billing requires separate audit
- **Real-time alerting:** This system operates on daily/weekly batch processing
- **Automatic enforcement:** No disconnection or penalty actions without human approval

### 7.3 Known Limitations

1. **Cold-start problem:** New customers with < 30 days of data cannot be reliably scored
2. **Sophisticated adversaries:** Gradual, small-scale theft may evade detection
3. **Data quality dependency:** Model performance degrades with poor meter data quality
4. **Label scarcity:** Supervised model performance limited by confirmed case volume

## 8. References and Further Reading

1. Glauner, P., et al. (2017). "The Challenge of Non-Technical Loss Detection Using Artificial Intelligence: A Survey." *International Journal of Computational Intelligence Systems*.

2. Jokar, P., et al. (2016). "Electricity Theft Detection in AMI Using Customers' Consumption Patterns." *IEEE Transactions on Smart Grid*.

3. World Bank (2019). "Reducing Technical and Non-Technical Losses in the Power Sector." *Energy Sector Management Assistance Program*.

4. Buzau, M., et al. (2019). "Detection of Non-Technical Losses Using Smart Meter Data and Supervised Learning." *IEEE Transactions on Smart Grid*.

5. European Union Agency for Cybersecurity (2023). "Smart Grid Security Certification." *ENISA Guidelines*.

---

**Document Control**

| Version | Date | Author | Changes |
|---------|------|--------|---------|
| 1.0 | 2024-12-15 | Data Science Team | Initial release |